In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Entry point of the graph.",
      "function_name": ""
    },
    "planner": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "Plan",
      "description": "Generates a step-by-step plan based on user input.",
      "function_name": "plan_step"
    },
    "agent": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "PlanExecute",
      "description": "Executes the first step of the plan and updates past steps.",
      "function_name": "execute_step"
    },
    "replan": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "Union[Response, Plan]",
      "description": "Updates the plan based on current state and past steps.",
      "function_name": "replan_step"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "End point of the graph.",
      "function_name": ""
    }
  },
  "edges": {
    "edge_1": {
      "source": "__START__",
      "target": "planner",
      "routing_conditions": "Start the planning process.",
      "conditional": false
    },
    "edge_2": {
      "source": "planner",
      "target": "agent",
      "routing_conditions": "After planning, execute the first step.",
      "conditional": false
    },
    "edge_3": {
      "source": "agent",
      "target": "replan",
      "routing_conditions": "After executing a step, check if replanning is needed.",
      "conditional": false
    },
    "edge_4": {
      "source": "replan",
      "target": "agent",
      "routing_conditions": "If no response is generated, continue to agent for further execution.",
      "conditional": true
    },
    "edge_5": {
      "source": "replan",
      "target": "__END__",
      "routing_conditions": "If a response is generated, end the process.",
      "conditional": true
    }
  }
}

# example with code snippets


In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "Type: None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The starting point of the graph.",
      "function_name": "",
      "code": ""
    },
    "supervisor": {
      "schema_info": "Router: type: TypedDict fields: - name: next type: Literal['food_logger', 'workout_logger', 'FINISH']",
      "input_schema": "MessagesState",
      "output_schema": "Router",
      "description": "This node manages the conversation and routes to the appropriate worker based on user input.",
      "function_name": "supervisor_node",
      "code": "def supervisor_node(state: State) -> Command[Literal[*members, '__end__']]:\n    messages = [\n        {'role': 'system', 'content': system_prompt},\n    ] + state['messages']\n    response = llm.with_structured_output(Router).invoke(messages)\n    goto = response['next']\n    if goto == 'FINISH':\n        goto = END\n\n    return Command(goto=goto, update={'next': goto})"
    },
    "food_logger": {
      "schema_info": "Type: MessagesState",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs food items and returns the result to the supervisor.",
      "function_name": "food_logging_node",
      "code": "def food_logging_node(state: State) -> Command[Literal['supervisor']]:\n    food_item = state['messages'][-1]['content']  # Get the last user input\n    result = food_logging_agent_tool.invoke(food_item)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result, name='food_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "workout_logger": {
      "schema_info": "Type: MessagesState",
      "input_schema": "MessagesState",
      "output_schema": "MessagesState",
      "description": "This node logs workout details and returns the result to the supervisor.",
      "function_name": "workout_logging_node",
      "code": "def workout_logging_node(state: State) -> Command[Literal['supervisor']]:\n    workout_details = state['messages'][-1]['content']  # Get the last user input\n    result = workout_analysis_tool.invoke(workout_details)\n    return Command(\n        update={\n            'messages': [\n                HumanMessage(content=result, name='workout_logger')\n            ]\n        },\n        goto='supervisor',\n    )"
    },
    "__END__": {
      "schema_info": "Type: None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The end point of the graph.",
      "function_name": "",
      "code": ""
    }
  },
  "edges": [
    { "source": "__START__", "target": "supervisor", "routing_condition": "Start the conversation by invoking the supervisor.", "conditional": false },
    { "source": "supervisor", "target": "food_logger", "routing_condition": "If the supervisor routes to food_logger based on user input.", "conditional": true },
    { "source": "supervisor", "target": "workout_logger", "routing_condition": "If the supervisor routes to workout_logger based on user input.", "conditional": true },
    { "source": "supervisor", "target": "__END__", "routing_condition": "If the supervisor routes to FINISH, end the conversation.", "conditional": false },
    { "source": "food_logger", "target": "supervisor", "routing_condition": "After logging food, return to the supervisor.", "conditional": false },
    { "source": "workout_logger", "target": "supervisor", "routing_condition": "After logging workout, return to the supervisor.", "conditional": false }
  ]
}

In [ ]:
{
  "nodes": {
    "__START__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The starting point of the graph.",
      "function_name": "start_node"
    },
    "supervisor": {
      "schema_info": "Router: TypedDict with next as a string literal of options.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node manages the conversation between workers and decides which worker to route to next.",
      "function_name": "supervisor_node"
    },
    "food_logger": {
      "schema_info": "State: TypedDict with messages as a list of messages.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs food intake and calculates calories.",
      "function_name": "food_logger_node"
    },
    "workout_tracker": {
      "schema_info": "State: TypedDict with messages as a list of messages.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs workout details and calculates calories burnt.",
      "function_name": "workout_tracker_node"
    },
    "__END__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The endpoint of the graph, indicating completion.",
      "function_name": "end_node"
    }
  },
  "edges": {
    { 
      "source": "__START__", 
      "target": "supervisor", 
      "routing_conditions": "Start the process by routing to the supervisor node.", 
      "conditional": false 
    },
    { 
      "source": "supervisor", 
      "target": "food_logger", 
      "routing_conditions": "If the supervisor decides to route to food_logger based on user input.", 
      "conditional": true 
    },
    { 
      "source": "supervisor", 
      "target": "workout_tracker", 
      "routing_conditions": "If the supervisor decides to route to workout_tracker based on user input.", 
      "conditional": true 
    },
    { 
      "source": "supervisor", 
      "target": "__END__", 
      "routing_conditions": "If the supervisor decides to finish the process.", 
      "conditional": true
    },
    { 
      "source": "food_logger", 
      "target": "supervisor", 
      "routing_conditions": "After logging food intake, return control to the supervisor.", 
      "conditional": false 
    },
    { 
      "source": "workout_tracker", 
      "target": "supervisor", 
      "routing_conditions": "After logging workout details, return control to the supervisor.", 
      "conditional": false 
    }
  }
}

In [ ]:
get_stock_price
send_email
read_email
get_current_weather